In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import os

In [ ]:
base_dir =  r'/content/car_and_bike/'
folders = os.listdir(base_dir)
print(folders)

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(256 , 256),
    batch_size= 32,
    class_mode='categorical',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(256, 256),
    batch_size= 32,
    class_mode='categorical',
    subset='validation')

In [ ]:
labels = (train_generator.class_indices)
labels = dict((v , k) for k , v in labels.items())
print(labels)

In [ ]:
base_model = tf.keras.applications.VGG19(input_shape=(256, 256,3),
                                               include_top=False,
                                               weights = "imagenet"
                                               )
base_model.trainable = False

In [ ]:
model_vgg = tf.keras.Sequential([
    base_model,
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(2, activation="softmax")
])

model_vgg.compile(loss='categorical_crossentropy' , optimizer=tf.keras.optimizers.Adam(lr = 0.001) , metrics='accuracy')

In [ ]:
model_vgg.summary()

In [ ]:
hist_vgg = model_vgg.fit(train_generator ,
          validation_data = validation_generator ,
          epochs = 10 ,
          steps_per_epoch = 3200/32, 
          validation_steps = 800/32, 
          )

In [ ]:
plt.plot(hist_vgg.history['accuracy'])
plt.plot(hist_vgg.history['val_accuracy'])
plt.title('Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'test'])
plt.show()

In [ ]:
plt.plot(hist_vgg.history['loss'])
plt.plot(hist_vgg.history['val_loss'])
plt.title('Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'test'])
plt.show()